In [2]:
import pandas as pd

In [4]:
# Upload de Datasets
users_ff = pd.read_csv('users_ff.csv')
users_ss1 = pd.read_csv('users_ss1.csv')
users_ss2 = pd.read_csv('users_ss2.csv')
users_ss3 = pd.read_csv('users_ss3.csv')
users_ts = pd.read_csv('users_ts.csv')
users_gen = pd.read_csv('users_gen.csv')
tweets_ff = pd.read_csv('tweets_ff.csv')
tweets_ss1 = pd.read_csv('tweets_ss1.csv')
tweets_ss2 = pd.read_csv('tweets_ss2.csv')
tweets_ss3 = pd.read_csv('tweets_ss3.csv')
tweets_ts = pd.read_csv('tweets_ts.csv')
tweets_gen = pd.read_csv('tweets_gen.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### Dataset Usuarios

In [7]:
# Agregar columna para categorizar
users_ss1['Cat'] = 'SocialSpambot'
users_ss2['Cat'] = 'SocialSpambot'
users_ss3['Cat'] = 'SocialSpambot'
users_ts['Cat'] = 'TraditionalSpambot'
users_ff['Cat'] = 'FollowersBot'
users_gen['Cat'] = 'Genuine'
# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
users_spam = pd.concat([users_ss1,users_ss2,users_ss3,users_ts,users_ff,users_gen],ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [8]:
users_spam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12737 entries, 0 to 12736
Data columns (total 43 columns):
Cat                                   12737 non-null object
contributors_enabled                  0 non-null float64
crawled_at                            9386 non-null object
created_at                            12737 non-null object
default_profile                       4404 non-null float64
default_profile_image                 66 non-null float64
description                           7011 non-null object
favourites_count                      12737 non-null int64
follow_request_sent                   0 non-null float64
followers_count                       12737 non-null int64
following                             0 non-null float64
friends_count                         12737 non-null int64
geo_enabled                           2346 non-null float64
id                                    12737 non-null int64
is_translator                         1 non-null float64
lang       

In [13]:
# Exportar a pkl
# users_spam.to_pickle('users_spam.pkl')

### Dataset Tweets

In [9]:
# Agregar columna para categorizar
tweets_ss1['Cat'] = 'SocialSpambot'
tweets_ss2['Cat'] = 'SocialSpambot'
tweets_ss3['Cat'] = 'SocialSpambot'
tweets_ts['Cat'] = 'TraditionalSpambot'
tweets_ff['Cat'] = 'FollowersBot'
tweets_gen['Cat'] = 'Genuine'
# Concatenar datasets de Usuarios (SocialSpambots & Traditional Spambots)
tweets_spam = pd.concat([tweets_ss1,tweets_ss2,tweets_ss3,tweets_ts,tweets_ff,tweets_gen],ignore_index=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [10]:
# Contar valores no nulos por columna
tweets_spam.notna().sum()

Cat                        6637616
contributors                     0
crawled_at                 6441588
created_at                 6637615
favorite_count             6637615
favorited                        0
geo                              0
id                         6637616
in_reply_to_screen_name    1039134
in_reply_to_status_id      6637615
in_reply_to_user_id        6637615
num_hashtags               6637615
num_mentions               6637615
num_urls                   6637615
place                       128651
possibly_sensitive           26812
reply_count                6637615
retweet_count              6637615
retweeted                        0
retweeted_status_id        6441588
source                     6637543
text                       6624609
timestamp                  6637615
truncated                      753
updated                    6441588
user_id                    6637615
dtype: int64

In [14]:
# Exportar a pkl
# tweets_spam.to_pickle('tweets_spam.pkl')